# Run REopt API

## Initialization

In [71]:
import pandas as pd
import json
import requests
from pathlib import Path
from datetime import datetime
from fuzzywuzzy import fuzz  # conda install fuzzywuzzy
from src.post_and_poll import get_api_results, get_run_uuid
from src.results_poller import poller
from API_KEYS import API_KEY, GOOGLE_API_KEY  # Get this private file from Gerry
import urllib3
urllib3.disable_warnings() # InsecureRequestWarning - need to add certificate
import asyncio
import tqdm.notebook as tqdm

In [47]:
inputs = Path('inputs')
outputs = Path('outputs')
ROOT_URL = "https://developer.nrel.gov/api/reopt/stable"

## Save data into spreadsheet
Extracts the API result into the GP spreadsheet

In [23]:
# First load the spreadsheet and visualize it as a sanity check
pd.set_option('display.max_columns', 100)
template = pd.read_excel(outputs / 'REopt_Master_2.xlsx', header=[0, 1, 2])
# template.iloc[12:13] # Uncomment this line to visualize

In [24]:
# Now extract the data from the json into a dataframe
OUTPUTS = template.columns[12:]  # which columns to edit
print('columns to edit are:', OUTPUTS.get_level_values(1))
def extract_info_into_df(data, rowi, api_response):
    # API documentation here:
    # https://github.com/NREL/REopt_API/blob/master/reo/nested_outputs.py
    site = api_response['outputs']['Scenario']['Site']
    pv = site['PV']
    financial = site['Financial']
    for o in OUTPUTS:
        data.loc[rowi, o] = None
    data.loc[rowi, OUTPUTS[0]] = pv['size_kw']
    data.loc[rowi, OUTPUTS[1]] = pv['average_yearly_energy_produced_kwh']
    data.loc[rowi, OUTPUTS[2]] = site['annual_total_renewable_energy_pct'] # total of electricity or energy?
    data.loc[rowi, OUTPUTS[3]] = site['lifecycle_emissions_reduction_CO2_pct']
    data.loc[rowi, OUTPUTS[4]] = site['lifecycle_emissions_cost_CO2'] - site['lifecycle_emissions_cost_CO2_bau']
    data.loc[rowi, OUTPUTS[5]] = site['lifecycle_emissions_cost_Health'] - site['lifecycle_emissions_cost_Health_bau']
    utility_cost = financial['lcc_us_dollars'] - financial['net_capital_costs_plus_om_us_dollars']
    data.loc[rowi, OUTPUTS[6]] = utility_cost - financial['lcc_bau_us_dollars']
    data.loc[rowi, OUTPUTS[7]] = financial['lcc_us_dollars']
    data.loc[rowi, OUTPUTS[8]] = financial['irr_pct']
    data.loc[rowi, OUTPUTS[9]] = site['lifecycle_emissions_tCO2'] - site['lifecycle_emissions_tCO2_bau']
    data.loc[rowi, OUTPUTS[10]] = site['lifecycle_emissions_tNOx'] - site['lifecycle_emissions_tNOx_bau']
    data.loc[rowi, OUTPUTS[11]] = site['lifecycle_emissions_tSO2'] - site['lifecycle_emissions_tSO2_bau']
    data.loc[rowi, OUTPUTS[12]] = site['lifecycle_emissions_tPM25'] - site['lifecycle_emissions_tPM25_bau']
    data.loc[rowi, OUTPUTS[13]] = "Virginia" # Completed By
    data.loc[rowi, OUTPUTS[14]] = datetime.now().strftime("%m/%d/%Y") # Date Completed

# Verify that the printed output "columns to edit" semantically matches with the code `data.loc[...] = ...`

columns to edit are: Index(['Recommended Solar Installation size (kW)',
       'Avg Annual PV Energy Produciton (kWh)',
       'Annual Renewable Electricity (% of total)',
       '% Reduction in CO2 Emissions from BAU',
       'Lifecycle Costs of Climate Emissions ($) (difference from BAU)',
       'Lifecycle Costs of Health Emissions ($) (difference from BAU)',
       'Total Utility Electricity Cost ($)', 'Total Life Cycle Costs ($)',
       'IRR', 'Lifecycle Total          t CO2 ', 'Lifecycle Total t Nox ',
       'Lifecycle Total t SO2 ', 'Lifecycle Total t PM2.5 ',
       'Lifecycle Total t PM2.5 ', 'Lifecycle Total t PM2.5 ',
       'Lifecycle Total t PM2.5 ', 'Lifecycle Total t PM2.5 '],
      dtype='object')


In [25]:
# Run a test - switch to `if True` to run this cell
if False:
    new_data = template.copy()                        # don't over-write the old data
    api_response = json.load(open(outputs / 'results' / '26900.json'))  # load the API results
    extract_info_into_df(new_data, 12, api_response)  # save the API results into the dataframe
    display(new_data.iloc[12:13])                     # Compare the new data...
    display(template.iloc[12:13])                     # to the old data

## URDB / metadata helper functions

First, fetch NREL URDB input

In [ ]:
# Test NREL URDB API
# https://apps.openei.org/services/doc/rest/util_rates/?version=3#request-url

def get_urdb_labelid(data_row):
    urdb_link = data_row[6]
    return urdb_link.split('/view/')[-1].split('#')[0]
def get_urdb_data(labelid):
    urdb_results = requests.get('https://api.openei.org/utility_rates',
                                params={
                                    'version': 'latest',
                                    'format': 'json',
                                    'api_key': API_KEY,
                                    'getpage': labelid,
                                    'detail': 'full',
                                })
    return urdb_results.json()['items'][0]

# Test
example_urdb_data = get_urdb_data(get_urdb_labelid(template.loc[12]))
print(str(example_urdb_data)[:200] + '...')

{'label': '5f4e57ce5457a3357aebd02a', 'uri': 'https://apps.openei.org/IURDB/rate/view/5f4e57ce5457a3357aebd02a', 'sector': 'Residential', 'energyweekendschedule': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...


In [ ]:
# Fetch lat/long from Google geocode:
def get_lat_long_from_google(city):
    res = requests.get('https://maps.googleapis.com/maps/api/geocode/json',
                       params={
                           'address': city,
                           'key': GOOGLE_API_KEY
                       }).json()
    loc = res['results'][0]['geometry']['location']
    return {'latitude': loc['lat'], 'longitude': loc['lng']}
lat_long = get_lat_long_from_google('Melbourne, FL')
print(lat_long)

{'latitude': 28.0836269, 'longitude': -80.60810889999999}


In [ ]:
# API documentation:
# https://github.com/NREL/REopt_API/blob/master/reo/nested_inputs.py
def create_input(city_name, net_metering_limit, urdb_id, lat_long=None):
    urdb = get_urdb_data(urdb_id)
    if lat_long is None:
        lat_long = get_lat_long_from_google(city_name)

    obj = {"Scenario": {"Site": {}}}
    site = obj["Scenario"]["Site"]
    # query = {"Scenario": {"Site":
    # {}},}
    # Step 1
    # TODO: energy goals
    # Step 2
    site["Storage"] = {"max_kw": "0.0", "max_kwh": "0.0"}
    # Step 3
    site["address"] = city_name.replace(',', '') + ' USA'
    site["latitude"] = lat_long['latitude']
    site["longitude"] = lat_long['longitude']
    site["ElectricTariff"] = {'urdb_response': urdb}
    #   2 - roofspace only
    site["land_acres"] = 0
    site["roof_squarefeet"] = None
    #   3. net metering system capacity
    site["ElectricTariff"]["net_metering_limit_kw"] = net_metering_limit
    #   5. Load profiles
    site["LoadProfile"] = {"doe_reference_name": "MidriseApartment"}
    # Step 6 - host effective tax rate
    site["Financial"] = {"offtaker_tax_pct": "0.0"}
    # Step 7
    obj["Scenario"]["include_climate_in_objective"] = False
    obj["Scenario"]["include_health_in_objective"] = False
    site["Financial"]["co2_cost_us_dollars_per_tonne"] = "185.0"
    site["net_metering_limit_kw"] = None
    site["PV"] = {
        "module_type": 1,  # Premium
        "dc_ac_ratio": "1.35",  # DC to AC size ratio
        "losses": "0.05",  # System losses
        "array_type": 1,  # Rooftop, fixed
        "federal_itc_pct": "0.3",  # Federal PV Incentives & Tax Treatment
    }
    return obj

post = create_input("Indianapolis, IN", 1000.0, '5f4e57ce5457a3357aebd02a')

## Run all data

In [52]:
# First test API to make sure it's running correctly.
api_response = get_api_results(post=post, 
                               API_KEY=API_KEY, 
                               api_url=ROOT_URL, 
                               results_file=outputs / "custom_input_test.json", 
                               run_id=None)
# new_data2 = template.copy()
# extract_info_into_df(new_data2, 34, api_response)
# display(new_data.iloc[34:35])                     # Compare the new data...
# display(template.iloc[34:35])                     # to the old data

main         INFO     Response OK from https://developer.nrel.gov/api/reopt/stable/job/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L.
main         INFO     Polling https://developer.nrel.gov/api/reopt/stable/job/223d6dd7-bd2b-459e-a294-dc1692a1a9b8/results/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L for results with interval of 5s...
main         INFO     Saved results to outputs/custom_input_test.json


In [53]:
json.dump(api_response, open(outputs / "custom_input_test.json", 'w'))

Now run on every row of spreadsheet

In [69]:
# First, generate all posts
posts = outputs / "posts"
posts.mkdir(exist_ok=True)

# print(template.columns[:11])
data = template.copy()
for _, row in data.iterrows():
    if pd.isna(row[0]):
        continue
    city = row[3] + ', ' + row[4]
    net_metering_capacity = row[11]
    urdb_id = get_urdb_labelid(row)

    post = create_input(city, net_metering_capacity, urdb_id=urdb_id)
    json.dump(post, open(posts / (city + '.json'), 'w'), indent=2)

In [70]:
# Now run all the queries!
# Create an NREL job for each row and save the uuid of the job (but don't wait for it to finish)
uuids = {}
for _, row in tqdm.tqdm(data.iterrows(), total=len(data)):
    if pd.isna(row[0]):
        continue
    city = row[3] + ', ' + row[4]
    post = json.load(open(posts / (city + '.json'), 'r'))

    # uuids[code] = get_run_uuid(post=post, API_KEY=API_KEY, api_url=ROOT_URL)
    resp = requests.post(ROOT_URL + '/job/?api_key=' + API_KEY, json=post)
    run_id = None
    if not resp.ok:
        raise RuntimeError("Status code {}. {}".format(resp.status_code, resp.content))
    else:
        uuids[city] = resp.json()['run_uuid']
display(uuids)

{'Atlanta, GA': '68fb21c3-d25c-4d5f-9d47-b9f83d048509', 'Augusta, GA': '2cdd00de-9d0c-4745-8154-8ef8a6fab2e5', 'Charleston, SC': '2d48088e-a9f3-47ab-be03-3c009e75a3ab', 'Charlotte, NC': '6757cffe-bcff-40e0-bd67-1c25a8b110c0', 'Chattanooga, TN': '7448c959-cac4-4e87-9eaf-ae68ea83bf92', 'Colorado Springs, CO': '64c8dd48-3088-437a-815f-dd691038d590', 'Columbia, SC': '8f1bd5a7-d56f-4b7b-b2a5-3d1224705d40', 'Des Moines, IA': 'd69f5ea2-e48e-4364-81f8-3bec93648eeb', 'Fayetteville, NC': '4c44acee-59d7-4660-a8d1-83e526cbb64c', 'Fayetteville, AR': '18101fb6-17ba-49c6-bc1d-eb1cecebd6b9', 'Greenville, SC': '125ec8c1-3527-4dec-88fd-5fd94aa92783', 'Huntsville, AL': 'be5bc4f1-90b8-4841-b40a-96c3bc5034ca', 'Indianapolis, IN': 'a9901074-ce2b-4dc0-b4e2-3eb8e10b8125', 'Knoxville, TN': 'd7f070a1-251b-412c-b4b4-3bb3251faa77', 'Lakeland, FL': '472a72df-4160-4a7a-9751-7e076fd2b6df', 'McAllen, TX': '2818b650-6518-4702-b82e-102bcc00e940', 'Myrtle Beach, SC': '8215a0e4-eb77-42dc-a73f-78ea8f9f4353', 'Nashville, T

In [72]:
# Now fetch the NREL results (once they're done)
json_results_folder = outputs / "results"
json_results_folder.mkdir(exist_ok=True)
for i, row in tqdm.tqdm(data.iterrows(), total=len(data)):
    if pd.isna(row[0]):
        continue
    city = row[3] + ', ' + row[4]

    api_response = get_api_results(post=post,
                                   API_KEY=API_KEY,
                                   api_url=ROOT_URL,
                                   results_file=json_results_folder / (city + '.json'),
                                   run_id=uuids[city])

    extract_info_into_df(data, i, api_response)

data.to_excel(outputs / "results.xlsx")

0it [00:00, ?it/s]

main         INFO     Polling https://developer.nrel.gov/api/reopt/stable/job/68fb21c3-d25c-4d5f-9d47-b9f83d048509/results/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L for results with interval of 5s...
main         INFO     Saved results to outputs/results/Atlanta, GA.json
main         INFO     Polling https://developer.nrel.gov/api/reopt/stable/job/2cdd00de-9d0c-4745-8154-8ef8a6fab2e5/results/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L for results with interval of 5s...
main         INFO     Saved results to outputs/results/Augusta, GA.json
main         INFO     Polling https://developer.nrel.gov/api/reopt/stable/job/2d48088e-a9f3-47ab-be03-3c009e75a3ab/results/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L for results with interval of 5s...
main         INFO     Saved results to outputs/results/Charleston, SC.json
main         INFO     Polling https://developer.nrel.gov/api/reopt/stable/job/6757cffe-bcff-40e0-bd67-1c25a8b110c0/results/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hl

In [77]:
# display(data[('Unnamed: 0_level_0', 'Point Person', 'Unnamed: 0_level_2')])
# display(data[data[('Unnamed: 0_level_0', 'Point Person', 'Unnamed: 0_level_2')] == 'Virginia'])
display(data)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                 cbsa       Point Person           Reviewer   
   Unnamed: 0_level_2 Unnamed: 1_level_2 Unnamed: 2_level_2   
0             12060.0              Grace                Cam   
1             12260.0              Julia                Cam   
2             16700.0              Julia           Virginia   
3             16740.0           Virginia              Grace   
4             16860.0              Grace              Julia   
5             17820.0                Cam              Julia   
6             17900.0                Cam           Virginia   
7             19780.0                Cam              Grace   
8             22180.0              Naomi           Virignia   
9             22220.0              Julia           Virginia   
10            24860.0           Virginia              Grace   
11            26620.0           Virginia                Cam   
12            26900.0           Virginia              Naomi   
13            28940.0                Cam           Virginia   
14            29460.0                Cam              Julia   
15            32580.0                Cam              Naomi   
16            34820.0              Julia                Cam   
17            34980.0              Grace              Julia   
18            35840.0                Cam              Julia   
19            36420.0                Cam              Grace   
20            37340.0              Naomi              Grace   
21            37860.0              Julia              Grace   
22            38860.0           Virginia              Julia   
23            38940.0              Grace              Naomi   
24            39580.0                Cam              Naomi   
25            40060.0                Cam           Virginia   
26            41700.0              Naomi                Cam   
27            44060.0              Naomi              Grace   
28            49180.0              Julia              Grace   
29                NaN                NaN                NaN   

   Unnamed: 3_level_0 Unnamed: 4_level_0  \
            City/MSA               State   
   Unnamed: 3_level_2 Unnamed: 4_level_2   
0             Atlanta                 GA   
1             Augusta                 GA   
2          Charleston                 SC   
3           Charlotte                 NC   
4         Chattanooga                 TN   
5    Colorado Springs                 CO   
6            Columbia                 SC   
7          Des Moines                 IA   
8        Fayetteville                 NC   
9        Fayetteville                 AR   
10         Greenville                 SC   
11         Huntsville                 AL   
12       Indianapolis                 IN   
13          Knoxville                 TN   
14           Lakeland                 FL   
15            McAllen                 TX   
16       Myrtle Beach                 SC   
17          Nashville                 TN   
18           Sarasota                 FL   
19      Oklahoma City                 OK   
20           Palm Bay                 FL   
21          Pensacola                 FL   
22           Portland                 ME   
23     Port St. Lucie                 FL   
24            Raleigh                 NC   
25           Richmond                 VA   
26        San Antonio                 TX   
27            Spokane                 WA   
28            Winston                 NC   
29                NaN                NaN   

                                               Inputs  \
                                      Utility (REopt)   
                                   Unnamed: 5_level_2   
0   Georgia Power Co: SCHEDULE TOU-REO-12 TIME OF ...   
1   Georgia Power Co Schedule TOU-REO-13 Time of U...   
2   South Carolina Electric & Gas Co: 5 (Residenti...   
3            Duke Energy Carolinas Residental Service   
4   City of Chattanooga, Tennessee (Utility Compan...   
5   City of Co

## Compare Hand-entered vs API results

In [117]:
data2 = template.copy()
new_headers = [(*col, 'manual') for col in template.columns]
data2.columns = pd.MultiIndex.from_tuples(new_headers)

for coli in range(len(template.columns) - 1, 11, -1):
    col = data.columns[coli]
    # newcol = (col[0], col[1], col[2], '(API)')
    # data2.insert(loc=0, column=(*col, 'manual'), value=template[col])
    data2.insert(loc=coli, column=(*col, 'diff'), value=data[col])
    data2.insert(loc=coli, column=(*col, 'API'), value=data[col])
# data2.insert(loc=0, column='NREL Results', value=None)
display(data2[:3])
data2.to_excel(outputs / "diff_results.xlsx")


Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                cbsa       Point Person           Reviewer          City/MSA    
  Unnamed: 0_level_2 Unnamed: 1_level_2 Unnamed: 2_level_2 Unnamed: 3_level_2   
              manual             manual             manual             manual   
0            12060.0              Grace                Cam            Atlanta   
1            12260.0              Julia                Cam            Augusta   
2            16700.0              Julia           Virginia         Charleston   

  Unnamed: 4_level_0                                             Inputs  \
               State                                    Utility (REopt)   
  Unnamed: 4_level_2                                 Unnamed: 5_level_2   
              manual                                             manual   
0                 GA  Georgia Power Co: SCHEDULE TOU-REO-12 TIME OF ...   
1                 GA  Georgia Power Co Schedule TOU-REO-13 Time of U...   
2                 SC  South Carolina Electric & Gas Co: 5 (Residenti...   

                                                                             \
  Utility Link (Rate Details from REopt) [OpenEI Link] System Capacity (kW)   
                                    Unnamed: 6_level_2   Unnamed: 7_level_2   
                                                manual               manual   
0  https://apps.openei.org/IURDB/rate/view/62d70d...                  10kW    
1  https://apps.openei.org/IURDB/rate/view/62d70d...                  10kW    
2  https://apps.openei.org/IURDB/rate/view/62c614...                  20 kW   

                                                                  \
  Adjusted System Capacity (kW) State system capacity limit (kW)   
             Unnamed: 8_level_2               Unnamed: 9_level_2   
                         manual                           manual   
0                          10.0                             10.0   
1                          10.0                             10.0   
2                          20.0                             20.0   

                                      \
  Min btw state and utility NEM (kW)   
                 Unnamed: 10_level_2   
                              manual   
0                                 10   
1                                 10   
2                                 20   

                                             \
  System capacity *100 (for 100 apartments)   
                        Unnamed: 11_level_2   
                                     manual   
0                                    1000.0   
1                                    1000.0   
2                                    2000.0   

              Energy Production & Fuel Use         \
  Recommended Solar Installation size (kW)          
                                       N/A          
                                       API manual   
0                                 198.4783  198.0   
1                                 194.1283  194.0   
2                                 194.8300  195.0   

                                                   \
  Avg Annual PV Energy Produciton (kWh)             
                              FINANCIAL             
                                    API    manual   
0                              285349.0  285349.0   
1                              285349.0  285349.0   
2                              285496.0  285496.0   

                                                    \
  Annual Renewable Electricity (% of total)          
                                  FINANCIAL          
                                        API manual   
0                                       1.0    1.0   
1                                       1.0    1.0   
2                                       1.0    1.0   

                      Climate Emissions           \
  % Reduction in CO2 Emissions from BAU            
                              FINANCIAL